In [1]:
import pandas as pd
import numpy as np
from late_fusion import *

In [2]:
retrieval = MultiModalLateFusionRetrieval()

feature_files = {
    #'mfcc_bow': 'dataset/id_mfcc_bow_mmsr.tsv',
    #'mfcc_stats': 'dataset/id_mfcc_stats_mmsr.tsv',

    'blf_correlation' : 'dataset/id_blf_correlation_mmsr.tsv',
    'blf_spectral': 'dataset/id_blf_spectral_mmsr.tsv',
    'blf_spectralcontrast': 'dataset/id_blf_spectralcontrast_mmsr.tsv',
    'blf_deltaspectral': 'dataset/id_blf_deltaspectral_mmsr.tsv',
    'blf_logfluc': 'dataset/id_blf_logfluc_mmsr.tsv',
    'blf_vardeltaspectral': 'dataset/id_blf_vardeltaspectral_mmsr.tsv',

    #'musicnn': 'dataset/id_musicnn_mmsr.tsv',

    #'lyrics_bert': 'dataset/id_lyrics_bert_mmsr.tsv',
    #'lyrics_tf-idf': 'dataset/id_lyrics_tf-idf_mmsr.tsv',
    #'lyrics_word2vec': 'dataset/id_lyrics_word2vec_mmsr.tsv',

    #'resnet': 'dataset/id_resnet_mmsr.tsv',
    #'vgg': 'dataset/id_vgg19_mmsr.tsv'
    #'incp': 'dataset/id_incp_mmsr.tsv'
}

retrieval.load_and_normalize_features(feature_files)

feature_weights = [
    #('mfcc_bow', 0.4),
    #('mfcc_stats', 0.4),

    ('blf_correlation', 0.4),
    ('blf_spectral', 0.4),
    ('blf_spectralcontrast', 0.4),
    ('blf_deltaspectral', 0.4),
    ('blf_logfluc', 0.4),
    ('blf_vardeltaspectral', 0.4),

    #('musicnn', 0.4),

    #('lyrics_bert', 0.4),
    #('lyrics_tf-idf', 0.4),
    #('lyrics_word2vec', 0.4),

    #('resnet', 0.4),
    #('vgg', 0.4),
    #('incp', 0.4)
]

infos = pd.read_csv("dataset/id_information_mmsr.tsv", sep="\t")

song_ids = infos['id'].tolist()
print(song_ids)
results = retrieval.batch_similarity_search(song_ids, top_k=10, fusion_method="average", weights=feature_weights)

['01rMxQv6vhyE1oQX', '02ZnlCGZEbkfCDxo', '04OjszRi9rC5BlHC', '04iitW3ffa0mhpx3', '04xUDjAYC14jsHyH', '06HvNTU9M9lnH71I', '06WySIW5UDIHF2Ri', '07bR4m5Oo4lvHJFW', '08dDspNuBDh8GOqV', '09mWmQm7ixF4cCjx', '0AUV4mCpGOuuWNUX', '0BVyGvdW4ntQDEcu', '0DMABfx807TVXBQO', '0DNKXcAGmMYmJdjP', '0DXkmUpDJT0AyerU', '0E1zWP8MXmbXfTTX', '0EVEKKlvBiB0Dkge', '0Ev84ceMs40159oJ', '0F5ITSxH0rBegwve', '0IqeNloQ3SuH1b4V', '0JYaNceX5H5sSUJi', '0K2vsBA9smDZxKno', '0KrfpITRYCMTHZno', '0KtUZOqkXcs5Q4Cn', '0LZdcvMJHDWZyVDS', '0MMDu3QYnVUgnX2L', '0MWOdRTVAx0staZz', '0MXDGjV7Qg5AEpGu', '0NTv0wWYCzQNjE0m', '0NaOCGE2Lkzbils9', '0NlLsUEZeTNagO3r', '0O17o38iEPkl4FWD', '0OuXKk7qlgvo6egK', '0PUlgI5adBJ6cFXK', '0QSQ3CDT5x4GRIgi', '0QSuci5nyuRojGbo', '0RswMLgk4KS4GbIP', '0SpykgFTUzfUTJkN', '0TanEa2QCnKAedsL', '0UHzuflmmfXZkZNQ', '0Vbfs25K0D6Yuq5T', '0Wc7JGEjjUDM4vp2', '0We9TRA5zyjS7ush', '0WjgTnhITQmzuoOc', '0WnFjxBFNLqMCT1i', '0XSKxF0xHVJYLAEi', '0Y5TcGlWJnW7IVdK', '0YtkHiVwDlsMHXy0', '0ZLbSoXXsKlXYZ7a', '0ZeUa6hsJJrS2bI4',

100%|██████████| 5148/5148 [50:39<00:00,  1.69it/s] 


In [4]:
results_df = pd.DataFrame(results)

In [7]:
results_df.transpose().to_csv('results/late_fusion_results.tsv', sep='\t', header=False)